In [24]:
import random
import pandas as pd
import numpy as np
from typing import Dict, Any, Tuple
from tabulate import tabulate

In [3]:
def simulateTradingDay(
    trade_outcomes: np.ndarray,  # ndarray of potential trade outcomes
    probabilities: np.ndarray,   # ndarray of corresponding probabilities
    daily_profit_target: float,
    drawdown_limit: float,
    initial_account_value: float,
    min_account_threshold: float,
    max_trades: int,
    flip_on_payout_met: bool,
    flip_value: float,
    payout_threshold: float
) -> Tuple[float, float, int]:
    """
    Simulates a single trading day for a futures strategy.

    Args:
        trade_outcomes: NumPy ndarray of potential trade outcomes (profit/loss in dollars).
        probabilities: NumPy ndarray of corresponding probabilities for each trade outcome. 
                        Must sum to 1 (or very close to 1).
        daily_profit_target: The daily profit target for the day.
        drawdown_limit: Maximum allowable drawdown for the day (percentage or dollars).
        initial_account_value: Starting account value for the day.
        min_account_threshold: Minimum allowable account value during the day.
        max_trades: Maximum number of trades allowed in a day.
        flip_on_payout_met: If True, apply the flip logic when the account value 
                            reaches or exceeds the payout_threshold within the day.
        flip_value: The fixed profit/loss amount applied in the flip logic 
                    (if flip_on_payout_met is True and flip_value is positive).
                    A positive value results in a 50/50 chance of a profit or loss.
        payout_threshold: The threshold for triggering a payout (used in flip logic).

    Returns:
        A tuple containing:
            - final_account_value: Account value at the end of the day.
            - daily_profit_loss: Total profit or loss for the day.
            - num_trades: Number of trades executed during the day. 
                          0 if a flip trade occurred.

    Raises:
        ValueError: If input values are invalid (e.g., negative account values, 
                    probabilities not summing to 1, drawdown_limit out of range).
    """

    # Input validation (ensure trade_outcomes and probabilities are ndarrays)
    if not isinstance(trade_outcomes, np.ndarray) or not isinstance(probabilities, np.ndarray):
        raise ValueError(
            "trade_outcomes and probabilities must be NumPy ndarrays.")

    # ... (Other input validation checks - similar to previous versions)

    account_value = initial_account_value
    daily_profit_loss = 0
    num_trades = 0

    if flip_value > 0 and flip_on_payout_met and account_value >= payout_threshold:
        # Apply flip logic if conditions are met
        num_trades = 0
        daily_profit_loss += flip_value if np.random.random() < 0.5 else -flip_value
        account_value += daily_profit_loss
    else:
        max_potential_trades = max_trades

        # Pre-generate trade outcomes for potential trades (using the provided ndarrays)
        all_trade_outcomes = np.random.choice(
            trade_outcomes,
            size=max_potential_trades,
            p=probabilities
        )

        # Calculate cumulative profit/loss and account values (using ndarrays)
        cumulative_profit_loss = np.cumsum(all_trade_outcomes)
        cumulative_account_values = initial_account_value + cumulative_profit_loss

        # Find the index where trading should stop
        stop_index = np.where(
            (cumulative_profit_loss >= daily_profit_target) |
            ((initial_account_value - cumulative_account_values) > drawdown_limit) |
            (cumulative_account_values < min_account_threshold) |
            (np.arange(max_potential_trades) >= max_trades - 1) |
            (flip_on_payout_met & (cumulative_account_values >= payout_threshold))
        )[0]

        if stop_index.size > 0:
            stop_index = stop_index[0]
        else:
            stop_index = max_potential_trades

        # Update final values based on the stopping point
        num_trades = stop_index + 1
        daily_profit_loss = cumulative_profit_loss[stop_index]
        account_value = cumulative_account_values[stop_index]

    return account_value, daily_profit_loss, num_trades

In [4]:
def simulateTradingDayPreNumpy(
    trade_outcomes,
    probabilities,
    daily_profit_target,
    drawdown_limit,
    initial_account_value,
    min_account_threshold,
    max_trades,
    flip_on_payout_met,
    flip_value,
    payout_threshold,
):
    """
    Simulates a single trading day for a futures strategy.

    Args:
        trade_outcomes: List of potential trade outcomes (profit/loss in dollars).
        probabilities: Corresponding list of probabilities for each trade outcome.
        daily_profit_target: The daily profit target for each trading day
        drawdown_limit: Daily drawdown limit (percentage or dollars).
        initial_account_value: Starting account value for the day.
        min_account_threshold: Minimum allowable account value.
        max_trades: Maximum number of trades allowed in a day.
        flip_on_payout_met: Boolean indicating whether to apply the flip logic when the payout threshold is met within a day
        flip_value: A numerical value used to determine a fixed profit/loss outcome 
                    if it's positive and flip_on_payout_met is True
        payout_threshold: The threshold for triggering a payout (used in flip logic)

    Returns:
        final_account_value: Account value at the end of the day.
        daily_profit_loss: Total profit or loss for the day.
        num_trades: Number of trades executed during the day (0 if a flip trade occurred)
    """

    account_value = initial_account_value
    daily_profit_loss = 0
    num_trades = 0

    if flip_value > 0 and flip_on_payout_met and account_value >= payout_threshold:
        # Apply flip logic if conditions are met
        num_trades = 0
        daily_profit_loss += flip_value if random.random() < 0.5 else -flip_value
        account_value += daily_profit_loss
    else:
        # Normal trading loop if flip conditions aren't met or flip_value is not positive
        while (
            daily_profit_loss < daily_profit_target
            and (initial_account_value - account_value) <= drawdown_limit
            and account_value >= min_account_threshold
            and num_trades < max_trades
        ):
            trade_outcome = random.choices(
                trade_outcomes, weights=probabilities)[0]
            account_value += trade_outcome
            daily_profit_loss += trade_outcome
            num_trades += 1

            # Check if we should stop trading due to flip_on_payout_met
            if flip_on_payout_met and account_value >= payout_threshold:
                break

    return account_value, daily_profit_loss, num_trades

In [16]:
def simulatePayoutPeriod(
    trade_outcomes: np.ndarray,  # ndarray of potential trade outcomes
    probabilities: np.ndarray,   # ndarray of corresponding probabilities
    daily_profit_target: float,
    payout_profit_target: float,
    min_trading_days: int,
    initial_account_value: float,
    min_account_threshold: float,
    max_trades: int,
    daily_drawdown_limit: float,
    is_first_payout_period: bool,
    flip_on_payout_met: bool,
    flip_value: float
) -> Tuple[float, float, bool, int, int, float, pd.DataFrame]:
    """
    Simulates a payout period for a futures strategy.

    Args:
        trade_outcomes: NumPy ndarray of potential trade outcomes (profit/loss in dollars).
        probabilities: NumPy ndarray of corresponding probabilities for each trade outcome. 
                        Must sum to 1 (or very close to 1).
        daily_profit_target: The daily profit target for each trading day.
        payout_profit_target: The account value target that triggers a payout.
        min_trading_days: Minimum number of trading days required for a payout.
        initial_account_value: Starting account value for the payout period.
        min_account_threshold: Minimum allowable account value.
        max_trades: Maximum number of trades allowed per day.
        daily_drawdown_limit: Maximum allowable drawdown within a single trading day.
        is_first_payout_period: Boolean indicating if this is the first payout period in the simulation.
        flip_on_payout_met: If True, apply the flip logic when the payout threshold is met within a day.
        flip_value: The fixed profit/loss amount applied in the flip logic 
                    (if flip_on_payout_met is True and flip_value is positive).
                    A positive value results in a 50/50 chance of a profit or loss.

    Returns:
        A tuple containing:
            - final_account_value: Account value at the end of the payout period.
            - avg_trades_per_day: Average number of trades executed per day 
                                  during the payout period (including flip days).
            - payout_achieved: True if the payout_profit_target was reached or exceeded, 
                               False otherwise.
            - total_trading_days: Total number of trading days in the payout period 
                                  (including flip days).
            - actual_trading_days: The number of trading days where actual trading occurred 
                                   (excluding flip days).
            - avg_trades_per_trading_day: The average number of trades per day on 
                                          actual trading days (excluding flip days).
            - df: DataFrame containing the daily results of the payout period.

    Raises:
        ValueError: If input values are invalid (e.g., negative account values, 
                    probabilities not summing to 1, drawdown_limit out of range).
    """

    # Input validation
    if not isinstance(trade_outcomes, np.ndarray) or not isinstance(probabilities, np.ndarray):
        raise ValueError(
            "trade_outcomes and probabilities must be NumPy ndarrays.")

    # ... (Other input validation checks - similar to previous versions,
    #      ensure initial_account_value, min_account_threshold, max_trades,
    #      payout_profit_target, min_trading_days, and daily_drawdown_limit are valid)

    account_value = initial_account_value
    total_trading_days = 0
    payout_complete = False
    total_trades = 0
    highest_account_value = initial_account_value
    freeze_min_account_threshold = False
    payout_achieved = account_value >= payout_profit_target

    actual_trading_days = 0
    num_actual_trades = 0

    # Data tracking
    payout_period_data = []

    while not payout_complete:
        daily_profit_loss = 0

        if (flip_on_payout_met and payout_achieved):
            # Apply flip logic for the rest of the payout period
            # since the payout target has been met
            num_trades = 1  # Consider this a trade for avg calculation even if it's a flip

            if np.random.random() < 0.5:
                daily_profit_loss += flip_value
            else:
                daily_profit_loss -= flip_value

            account_value += daily_profit_loss
        else:
            account_value, daily_profit_loss, num_trades = simulateTradingDay(
                trade_outcomes,  # Pass ndarrays directly
                probabilities,
                daily_profit_target,
                daily_drawdown_limit,
                account_value,
                min_account_threshold,
                max_trades,
                flip_on_payout_met,
                flip_value,
                payout_profit_target
            )

            # Increment num_actual_trades and actual_trading_days
            num_actual_trades += num_trades
            actual_trading_days += 1

        total_trading_days += 1
        total_trades += num_trades

        # Record data for this trading day
        payout_period_data.append({
            'Day': total_trading_days,
            'Number of Trades': num_trades,
            'Daily Profit/Loss': daily_profit_loss,
            'Account Value': account_value
        })

        if not payout_achieved:
            payout_achieved = account_value >= payout_profit_target

        # Check if payout is complete
        payout_complete = (
            account_value < min_account_threshold
            or (
                total_trading_days >= min_trading_days
                and account_value >= payout_profit_target
            )
        )

        # Update min_account_threshold if this is the first payout period
        if is_first_payout_period and not freeze_min_account_threshold and account_value > highest_account_value:
            delta = account_value - highest_account_value
            highest_account_value = account_value
            min_account_threshold = min(0, min_account_threshold + delta)
            freeze_min_account_threshold = min_account_threshold == 0

    if total_trading_days > 0:
        avg_trades_per_day = total_trades / total_trading_days
    else:
        avg_trades_per_day = 0

    # Calculate average trades per actual trading day
    if actual_trading_days > 0:
        avg_trades_per_trading_day = num_actual_trades / actual_trading_days
    else:
        avg_trades_per_trading_day = 0

    # Create and print DataFrame
    df = pd.DataFrame(payout_period_data)
    # if len(df) <= 20:
    # print("\n" + df.to_string(index=False) + "\n")

    return account_value, avg_trades_per_day, payout_achieved, total_trading_days, actual_trading_days, avg_trades_per_trading_day, df

In [9]:
def sampleSimulatePayoutPeriodCall():
    # Example parameter values (adjust these to match your strategy and preferences)
    tick_size = 12.5
    q1 = 0
    q2 = 4
    q3 = 2
    win_multiplier = 4
    loss_multiplier = 2.5

    full_win = q1 * 1 * tick_size + q2 * 2 * tick_size + q3 * 2 * tick_size
    partial_1 = q1 * 1 * tick_size + q2 * -5 * tick_size + q3 * -5 * tick_size
    full_loss = q1 * -8 * tick_size + q2 * -8 * tick_size + q3 * -8 * tick_size

    # Convert trade_outcomes and probabilities to ndarrays
    trade_outcomes = np.array([full_win, partial_1, full_loss])
    probabilities = np.array([0.81, 0.05, 0.14])

    daily_profit_target = win_multiplier * full_win
    payout_profit_target = 10500  # Aiming for a $10,000 account value
    min_trading_days = 10
    initial_account_value = 7500
    min_account_threshold = 0  # Allow for an initial loss of up to $2000
    max_trades = 6
    daily_drawdown_limit = loss_multiplier * -full_loss
    is_first_payout_period = False
    flip_on_payout_met = True
    flip_value = tick_size

    print(trade_outcomes, daily_profit_target, daily_drawdown_limit)

    # Call the function
    final_account_value, avg_trades_per_day, payout_achieved, total_trading_days, actual_trading_days, avg_trades_per_trading_day, df = simulatePayoutPeriod(
        trade_outcomes,  # Pass as ndarray
        probabilities,   # Pass as ndarray
        daily_profit_target,
        payout_profit_target,
        min_trading_days,
        initial_account_value,
        min_account_threshold,
        max_trades,
        daily_drawdown_limit,
        is_first_payout_period,
        flip_on_payout_met,
        flip_value
    )

    # Print the results
    print("\n*** Payout Period Results ***")
    print(f"Final Account Value: ${final_account_value:.2f}")
    print(f"Average Trades per Day: {avg_trades_per_day:.2f}")
    print(f"Payout Achieved: {payout_achieved}")
    print(f"Total Trading Days: {total_trading_days:.0f}")
    print(
        f"Actual Trading Days (excluding flip days): {actual_trading_days:.0f}")
    print(
        f"Average Trades per Actual Trading Day: {avg_trades_per_trading_day:.2f}")


sampleSimulatePayoutPeriodCall()

[ 150. -375. -600.] 600.0 1500.0

 Day  Number of Trades  Daily Profit/Loss  Account Value
   1                 6              150.0         7650.0
   2                 6              375.0         8025.0
   3                 4              600.0         8625.0
   4                 4              600.0         9225.0
   5                 4              600.0         9825.0
   6                 6              150.0         9975.0
   7                 6             -375.0         9600.0
   8                 6              150.0         9750.0
   9                 4              600.0        10350.0
  10                 6            -1125.0         9225.0
  11                 4              600.0         9825.0
  12                 4              600.0        10425.0
  13                 1              150.0        10575.0


*** Payout Period Results ***
Final Account Value: $10575.00
Average Trades per Day: 4.69
Payout Achieved: True
Total Trading Days: 13
Actual Trading Days (excluding 

In [47]:
def simulateEvaluation(
    trade_outcomes: np.ndarray,
    probabilities: np.ndarray,
    daily_profit_target: float,
    payout_profit_target: float,
    min_trading_days: int,
    initial_account_value: float,
    min_account_threshold: float,
    max_trades: int,
    daily_drawdown_limit: float,
    flip_on_payout_met: bool,
    flip_value: float,
    payouts_to_achieve: int,
    payout_amount: float
) -> Tuple[float, int, bool, float, float, float, float, float, pd.DataFrame]:
    """
    Simulates multiple payout periods to evaluate a trading strategy.

    Args:
        trade_outcomes: NumPy ndarray of potential trade outcomes (profit/loss in dollars).
        probabilities: NumPy ndarray of corresponding probabilities for each trade outcome. 
                        Must sum to 1 (or very close to 1).
        daily_profit_target: The daily profit target for each trading day.
        payout_profit_target: The initial account value target that triggers a payout.
        min_trading_days: Minimum number of trading days required for a payout.
        initial_account_value: The starting account value for the evaluation.
        min_account_threshold: The minimum allowable account value. 
                               If the account value falls below this threshold, 
                               the evaluation ends.
        max_trades: Maximum number of trades allowed per day.
        daily_drawdown_limit: Maximum allowable drawdown within a single trading day.
        flip_on_payout_met: If True, apply the flip logic when the payout threshold is met within a day.
        flip_value: The fixed profit/loss amount applied in the flip logic 
                    (if flip_on_payout_met is True and flip_value is positive).
                    A positive value results in a 50/50 chance of a profit or loss.
        payouts_to_achieve: The target number of successful payouts to achieve 
                            during the evaluation.
        payout_amount: The amount to be withdrawn from the account value 
                       after each successful payout.

    Returns:
        A tuple containing:
            - final_account_value: The account value at the end of the evaluation.
            - total_payouts: The total number of successful payouts achieved.
            - evaluation_succeeded: True if the target number of payouts was achieved, 
                                    False otherwise.
            - avg_trades_per_day: The average number of trades per day across all payout periods
                                  (including flip days).
            - avg_total_trading_days: The average number of total trading days per payout period
                                      (including flip days).
            - avg_actual_trading_days: The average number of actual trading days per payout period
                                       (excluding flip days).
            - avg_avg_trades_per_trading_day: The average of the average number of trades per trading day
                                              across all payout periods (excluding flip days).
            - total_amount_paid_out: The total amount paid out during the evaluation.
    """

    # Input validation (ensure trade_outcomes and probabilities are ndarrays)
    if not isinstance(trade_outcomes, np.ndarray) or not isinstance(probabilities, np.ndarray):
        raise ValueError(
            "trade_outcomes and probabilities must be NumPy ndarrays.")

    # ... (Other input validation checks - similar to previous versions,
    #      ensure initial_account_value, min_account_threshold, max_trades,
    #      payout_profit_target, min_trading_days, daily_drawdown_limit, and
    #      payout_amount are valid)

    account_value = initial_account_value
    total_payouts = 0

    total_avg_trades_per_day = 0
    total_total_trading_days = 0
    total_actual_trading_days = 0
    total_avg_trades_per_trading_day = 0
    total_amount_paid_out = 0

    payout_period_tracking_data = []
    evaluation_data = []

    while total_payouts < payouts_to_achieve and account_value >= min_account_threshold:
        payout_period_tracking_data.append({
            'Payout Period': total_payouts,
            'Account Value': account_value,
            'Payout Profit Target': payout_profit_target,
            'Minimum Account Threshold': min_account_threshold
        })

        # Pass all necessary parameters to simulatePayoutPeriod
        (
            account_value,
            pp_avg_trades_per_day,  # Use different variable names for return values
            payout_achieved,
            pp_total_trading_days,
            pp_actual_trading_days,
            pp_avg_trades_per_trading_day,
            _
        ) = simulatePayoutPeriod(
            trade_outcomes,
            probabilities,
            daily_profit_target,
            payout_profit_target,
            min_trading_days,
            account_value,
            min_account_threshold,
            max_trades,
            daily_drawdown_limit,
            total_payouts == 0,
            flip_on_payout_met,
            flip_value,
        )

        # Print returned values from simulatePayoutPeriod
        print(f"\n  pp_avg_trades_per_day: {pp_avg_trades_per_day:.2f}")
        print(f"  pp_total_trading_days: {pp_total_trading_days}")
        print(f"  pp_actual_trading_days: {pp_actual_trading_days}")
        print(
            f"  pp_avg_trades_per_trading_day: {pp_avg_trades_per_trading_day:.2f}")
        print(f"  payout_achieved: {payout_achieved}")

        if payout_achieved:
            if total_payouts == 0:
                min_account_threshold = 0

            # Accumulate values for averaging (including the first payout)
            if total_payouts == 1:
                total_avg_trades_per_day = pp_avg_trades_per_day
                total_total_trading_days = pp_total_trading_days
                total_actual_trading_days = pp_actual_trading_days
                total_avg_trades_per_trading_day = pp_avg_trades_per_trading_day
            else:
                total_avg_trades_per_day += pp_avg_trades_per_day
                total_total_trading_days += pp_total_trading_days
                total_actual_trading_days += pp_actual_trading_days
                total_avg_trades_per_trading_day += pp_avg_trades_per_trading_day

            # Print accumulated totals
            print(
                f"\n  total_avg_trades_per_day: {total_avg_trades_per_day:.2f}")
            print(
                f"  total_total_trading_days: {total_total_trading_days:.2f}")
            print(
                f"  total_actual_trading_days: {total_actual_trading_days:.2f}")
            print(
                f"  total_avg_trades_per_trading_day: {total_avg_trades_per_trading_day:.2f}")

            # Update payout_profit_target, withdraw payout_amount, and track total paid out
            payout_profit_target = account_value
            account_value -= payout_amount
            total_amount_paid_out += payout_amount

            total_payouts += 1
        elif total_payouts == 0:  # Added condition to break if the first payout fails
            break

        # Record data for this payout period
        evaluation_data.append({
            'Payout Period': total_payouts,  # 1-indexed for user readability
            'Account Value': account_value,
            'Payout Achieved': payout_achieved,
            'Total Trading Days': pp_total_trading_days
        })

    evaluation_succeeded = total_payouts == payouts_to_achieve

    # Calculate averages, excluding the first payout if there were multiple
    avg_trades_per_day = 0
    avg_total_trading_days = 0
    avg_actual_trading_days = 0
    avg_avg_trades_per_trading_day = 0

    print("\n*** Final Averages ***")
    # Calculate averages, excluding the first payout if there were multiple
    if total_payouts > 1:
        print(f"divisor: {total_payouts - 1}")
        avg_trades_per_day = total_avg_trades_per_day / total_payouts - 1
        avg_total_trading_days = total_total_trading_days / total_payouts - 1
        avg_actual_trading_days = total_actual_trading_days / total_payouts - 1
        avg_avg_trades_per_trading_day = total_avg_trades_per_trading_day / total_payouts - 1
    else:
        # If only one or zero payouts were achieved, use the accumulated totals directly (or 0 if none)
        avg_trades_per_day = total_avg_trades_per_day if total_payouts > 0 else 0
        avg_total_trading_days = total_total_trading_days if total_payouts > 0 else 0
        avg_actual_trading_days = total_actual_trading_days if total_payouts > 0 else 0
        avg_avg_trades_per_trading_day = total_avg_trades_per_trading_day if total_payouts > 0 else 0

    # Print final averages

    print(f"avg_trades_per_day: {avg_trades_per_day:.2f}")
    print(f"avg_total_trading_days: {avg_total_trading_days:.2f}")
    print(f"avg_actual_trading_days: {avg_actual_trading_days:.2f}")
    print(
        f"avg_avg_trades_per_trading_day: {avg_avg_trades_per_trading_day:.2f}")

    df_evaluation = pd.DataFrame(evaluation_data)
    df_payout_period_tracking = pd.DataFrame(payout_period_tracking_data)

    # Print the payout_period_tracking DataFrame
    print(tabulate(df_payout_period_tracking, headers='keys',
          tablefmt='psql', showindex=False, floatfmt=".2f"))

    return (
        account_value,
        total_payouts,
        evaluation_succeeded,
        avg_trades_per_day,
        avg_total_trading_days,
        avg_actual_trading_days,
        avg_avg_trades_per_trading_day,
        total_amount_paid_out,
        df_evaluation
    )

In [56]:
def sampleSimulateEvaluationCall():
    # Example parameter values (reusing from sampleSimulatePayoutPeriodCall)
    tick_size = 12.5
    q1 = 0
    q2 = 0
    q3 = 6
    win_multiplier = 4
    loss_multiplier = 2.5

    full_win = q1 * 1 * tick_size + q2 * 2 * tick_size + q3 * 2 * tick_size
    partial_1 = q1 * 1 * tick_size + q2 * -5 * tick_size + q3 * -5 * tick_size
    full_loss = q1 * -8 * tick_size + q2 * -8 * tick_size + q3 * -8 * tick_size

    trade_outcomes = np.array([full_win, partial_1, full_loss])
    probabilities = np.array([0.81, 0.05, 0.14])
    daily_profit_target = win_multiplier * full_win
    payout_profit_target = 10500
    min_trading_days = 10
    initial_account_value = 0
    min_account_threshold = -7500
    max_trades = 6
    daily_drawdown_limit = loss_multiplier * -full_loss
    flip_on_payout_met = True
    flip_value = tick_size

    # Additional parameters specific to simulateEvaluation
    payouts_to_achieve = 3  # Example: Aim to achieve 3 payouts
    payout_amount = 3000     # Example: Withdraw $5000 after each successful payout

    print(trade_outcomes, daily_profit_target, daily_drawdown_limit)

    # Call simulateEvaluation
    (
        final_account_value,
        total_payouts,
        evaluation_succeeded,
        avg_trades_per_day,
        avg_total_trading_days,
        avg_actual_trading_days,
        avg_avg_trades_per_trading_day,
        total_amount_paid_out,
        df_evaluation
    ) = simulateEvaluation(
        trade_outcomes,
        probabilities,
        daily_profit_target,
        payout_profit_target,
        min_trading_days,
        initial_account_value,
        min_account_threshold,
        max_trades,
        daily_drawdown_limit,
        flip_on_payout_met,
        flip_value,
        payouts_to_achieve,
        payout_amount
    )

    # Print the results
    print("\n*** Evaluation Results ***")
    print(f"Final Account Value: ${final_account_value:.2f}")
    print(f"Total Payouts Achieved: {total_payouts}")
    print(f"Total Amount Paid Out: ${total_amount_paid_out:.2f}")
    print(f"Evaluation Succeeded: {evaluation_succeeded}")
    print("\n*** Averages Across Payout Periods ***")
    print(f"Average Trades per Day: {avg_trades_per_day:.2f}")
    print(f"Average Total Trading Days: {avg_total_trading_days:.2f}")
    print(f"Average Actual Trading Days: {avg_actual_trading_days:.2f}")
    print(
        f"Average Trades per Actual Trading Day: {avg_avg_trades_per_trading_day:.2f}")
    print(f"Total Amount Paid Out: ${total_amount_paid_out:.2f}")

    print("\n*** Payout Period Data ***")
    print(tabulate(df_evaluation, headers='keys',
          tablefmt='psql', showindex=False, floatfmt=".2f"))


# Call the function to run the sample evaluation
sampleSimulateEvaluationCall()

[ 150. -375. -600.] 600.0 1500.0

  pp_avg_trades_per_day: 5.17
  pp_total_trading_days: 159
  pp_actual_trading_days: 159
  pp_avg_trades_per_trading_day: 5.17
  payout_achieved: True

  total_avg_trades_per_day: 5.17
  total_total_trading_days: 159.00
  total_actual_trading_days: 159.00
  total_avg_trades_per_trading_day: 5.17

  pp_avg_trades_per_day: 4.47
  pp_total_trading_days: 19
  pp_actual_trading_days: 19
  pp_avg_trades_per_trading_day: 4.47
  payout_achieved: True

  total_avg_trades_per_day: 4.47
  total_total_trading_days: 19.00
  total_actual_trading_days: 19.00
  total_avg_trades_per_trading_day: 4.47

  pp_avg_trades_per_day: 5.15
  pp_total_trading_days: 52
  pp_actual_trading_days: 52
  pp_avg_trades_per_trading_day: 5.15
  payout_achieved: True

  total_avg_trades_per_day: 9.63
  total_total_trading_days: 71.00
  total_actual_trading_days: 71.00
  total_avg_trades_per_trading_day: 9.63

*** Final Averages ***
divisor: 2
avg_trades_per_day: 2.21
avg_total_trading_day

In [14]:
def optimize(
    param_space, num_simulations, payouts_to_achieve, initial_account_value, min_account_threshold, success_probability_threshold, acq_func="EI", n_calls=100, n_random_starts=10, n_jobs=-1, top_n_results=5, value_weight=0.75, probability_weight=0.25, maximize_value=True
):
    """
    Optimizes trading strategy parameters using nested Monte Carlo simulations and Bayesian optimization.

    Args:
        param_space: A dictionary where keys are parameter names and values are tuples 
                     (lower_bound, upper_bound) defining the search space for each parameter.
        num_simulations: The number of Monte Carlo simulations (evaluations) to run for each parameter set.
        payouts_to_achieve: The target number of payouts to achieve in each evaluation.
        initial_account_value: The starting account value for each evaluation.
        min_account_threshold: The minimum allowable account value. 
                               If the account value falls below this threshold during an evaluation, 
                               it is considered unsuccessful.
        success_probability_threshold: The minimum required probability of success 
                                       (achieving the target number of payouts) for a 
                                       parameter set to be considered in the final results.
        acq_func: The acquisition function used for Bayesian optimization (default: "EI").
        n_calls: The maximum number of iterations for the Bayesian optimization process (default: 100).
        n_random_starts: The number of random initialization points for Bayesian optimization (default: 10).
        n_jobs: The number of parallel jobs to run for Bayesian optimization.
                -1 means using all available cores (default: -1).
        top_n_results: The number of top-performing parameter sets to return (default: 5).
        value_weight: The weight assigned to the average final value objective in the 
                      combined objective function (0-1, default: 0.75).
        probability_weight: The weight assigned to the probability of success objective 
                            in the combined objective function (0-1, default: 0.25).
        maximize_value: If True, the optimization aims to maximize the average final value. 
                        If False, it aims to minimize it (default: True).

    Returns:
        A list of tuples, each representing a top-performing parameter set. 
        Each tuple contains:
            - params: A dictionary where keys are parameter names and values are the optimized values.
            - avg_final_value: The average final account value achieved with these parameters 
                               across multiple simulations.
            - probability_of_success: The probability of achieving the target number of payouts 
                                      with these parameters.

    """

    all_final_values = []

    @use_named_args(dimensions=[Real(low, high) for low, high in param_space.values()])
    def objective_function(**params):
        # Run multiple evaluations with the same parameters to get an average
        final_values = []
        for _ in range(num_simulations):
            final_value, _ = simulateEvaluation(
                params, payouts_to_achieve, initial_account_value, min_account_threshold)
            final_values.append(final_value)
        avg_final_value = sum(final_values) / len(final_values)

        # Calculate probability of meeting payouts_to_achieve goal
        successful_evaluations = sum(
            1 for final_value in final_values if final_value >= min_account_threshold)
        probability_of_success = successful_evaluations / num_simulations

        all_final_values.extend(final_values)

        # Return average final value, possibly negated for minimization
        return -avg_final_value if maximize_value else avg_final_value, -probability_of_success

    opt = Optimizer(
        dimensions=[Real(low, high) for low, high in param_space.values()],
        acq_func=acq_func,
        n_initial_points=n_random_starts,
    )

    for _ in range(n_calls):
        next_params = opt.ask()
        result = objective_function(
            **{dim.name: val for dim, val in zip(opt.space.dimensions, next_params)})
        opt.tell(next_params, result)

        # Check if the success probability meets the threshold
        if -result[1] >= success_probability_threshold:
            break

    # Standardize final values
    mean_final_value = np.mean(all_final_values)
    std_final_value = np.std(all_final_values)

    # Get top N results from the optimizer
    top_results = []
    for point, value in zip(opt.Xi, opt.yi):
        params = {dim.name: x for dim, x in zip(opt.space.dimensions, point)}
        avg_final_value, probability_of_success = (
            -value[0] if maximize_value else value[0]), -value[1]
        if probability_of_success >= success_probability_threshold:
            # Standardize the average final value
            normalized_value = (
                avg_final_value - mean_final_value) / std_final_value

            top_results.append(
                (params, normalized_value, probability_of_success))

    # Sort by weighted objective, considering maximization/minimization
    top_results = sorted(
        top_results,
        key=lambda x: (value_weight * x[1] if maximize_value else -
                       value_weight * x[1]) + probability_weight * x[2],
        reverse=True,
    )[:top_n_results]

    # Denormalize values for the top results
    top_results = [
        (result[0], result[1] * std_final_value + mean_final_value, result[2])
        for result in top_results
    ]

    return top_results

In [16]:
def initialize_param_space():
    """
    Initializes the parameter space for the trading strategy optimization.

    Returns:
        A dictionary where keys are parameter names and values are tuples of 
        (lower_bound, upper_bound) or (lower_bound, upper_bound, step) representing 
        the range of possible values for each parameter.

    Parameter Descriptions:
        - trade_outcomes: Nested lists representing the possible profit/loss outcomes for trades.
                          Each inner list defines the range [low, high] for one outcome.
        - probabilities: Nested lists representing the probabilities associated with each trade outcome.
                         Each inner list defines the range [low, high] for the probabilities of the 
                         corresponding outcomes in `trade_outcomes`. The probabilities within each 
                         inner list should sum to 1.
        - daily_profit_target: The daily profit target for each trading day.
        - payout_profit_target: The account value target that triggers a payout.
        - min_trading_days: The minimum number of trading days required for a payout.
        - max_trades: The maximum number of trades allowed per day.
        - daily_drawdown_limit: The maximum allowable drawdown (loss) within a single trading day.
        - flip_on_payout_met: Whether to apply the "flip" logic when the payout target is met within a day.
        - flip_value: The fixed profit/loss amount applied in the "flip" logic (if `flip_on_payout_met` is True).
    """

    param_space = {
        # Nested lists: [low, high] for each outcome
        "trade_outcomes": ([100, -200], [300, -500]),
        # Nested lists: [low, high] for each probability (must sum to 1 within each list)
        "probabilities": ([0.5, 0.2, 0.3], [0.9, 0.05, 0.05]),
        "daily_profit_target": (100, 1000, 50),  # step value of 50
        "payout_profit_target": (5000, 20000),
        "min_trading_days": (3, 10),
        "max_trades": (1, 10),
        "daily_drawdown_limit": (500, 5000),
        "flip_on_payout_met": (True, False),  # Boolean parameter
        "flip_value": (-1000, 1000)
    }

    return param_space

In [17]:
def display_results(top_results):
    """
    Displays the top optimization results in an HTML table and generates bar charts 
    to visualize the average final value and probability of success for each parameter set.

    Args:
        top_results: A list of tuples, each containing:
            - params: A dictionary where keys are parameter names and values are the optimized values.
            - avg_final_value: The average final account value achieved with these parameters.
            - probability_of_success: The probability of achieving the target number of payouts 
                                      with these parameters.

    Returns:
        An HTML object containing the formatted table and charts for display.
    """

    # Create table data
    headers = ["Parameters", "Avg. Final Value", "Probability of Success"]
    table_data = [
        [str(params), f"${avg_final_value:.2f}",
         f"{probability_of_success:.2%}"]
        for params, avg_final_value, probability_of_success in top_results
    ]

    # Generate table HTML
    table_html = tabulate(table_data, headers, tablefmt="html")

    # Generate charts
    # Example: bar chart for avg_final_value
    param_names = [str(params) for params, _, _ in top_results]
    avg_final_values = [avg_final_value for _,
                        avg_final_value, _ in top_results]
    probabilities = [probability_of_success for _,
                     _, probability_of_success in top_results]

    plt.figure(figsize=(10, 6))
    plt.bar(param_names, avg_final_values, color='skyblue')
    plt.xlabel('Parameter Set')
    plt.ylabel('Average Final Value ($)')
    plt.title('Top Parameter Sets by Average Final Value')
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Save the chart to a temporary file
    plt.savefig('avg_final_value_chart.png')
    plt.close()

    # Generate chart HTML for avg_final_value
    chart1_html = f'<img src="avg_final_value_chart.png" alt="Avg Final Value Chart">'

    # Example: bar chart for probabilities
    plt.figure(figsize=(10, 6))
    plt.bar(param_names, probabilities, color='lightgreen')
    plt.xlabel('Parameter Set')
    plt.ylabel('Probability of Success')
    plt.title('Top Parameter Sets by Probability of Success')
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Save the chart to a temporary file
    plt.savefig('probability_chart.png')
    plt.close()

    # Generate chart HTML for probabilities
    chart2_html = f'<img src="probability_chart.png" alt="Probability Chart">'

    # Combine table and chart HTML
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
    <title>Optimization Results</title>
    </head>
    <body>
    <h1>Optimization Results</h1>
    {table_html}
    <h2>Average Final Value</h2>
    {chart1_html}
    <h2>Probability of Success</h2>
    {chart2_html}
    </body>
    </html>
    """

    # Display the HTML content
    return HTML(html_content)

In [18]:
def run_optimization_with_weights(
    param_space,
    num_simulations,
    payouts_to_achieve,
    initial_account_value,
    min_account_threshold,
    success_probability_threshold,
    weight_combinations,
    acq_func="EI",
    n_calls=100,
    n_random_starts=10,
    n_jobs=-1,
    top_n_results=5,
    maximize_value=True,
):
    """
    Runs the optimization process with multiple weight combinations for the objective function 
    and displays the results for each combination.

    Args:
        # ... (Same arguments as in the `optimize` function)
        weight_combinations: A list of tuples, where each tuple represents a combination 
                             of weights for the average final value and the probability 
                             of success objectives. Each tuple should have the format:
                             (value_weight, probability_weight), where both weights are 
                             between 0 and 1, and their sum is 1.

    """

    for value_weight, probability_weight in weight_combinations:
        print(
            f"\n--- Optimization with value_weight={value_weight}, probability_weight={probability_weight} ---\n")

        top_results = optimize(
            param_space,
            num_simulations,
            payouts_to_achieve,
            initial_account_value,
            min_account_threshold,
            success_probability_threshold,
            acq_func,
            n_calls,
            n_random_starts,
            n_jobs,
            top_n_results,
            value_weight,
            probability_weight,
            maximize_value,
        )

        display_results(top_results)